In [ ]:
import qcodes as qc
import numpy as np
from time import sleep

import sweep 
from barreralabdrivers.utils.param_utils import paramp
from cappy.config import coolconfig, datapath
from qcodes.instrument_drivers.QuantumDesign import DynaCool

In [ ]:
# instrument connections 
qc.Instrument.close_all()   # in case of existing connecitons
container = qc.Station(config_file=str(coolconfig))

acdac = container.load_instrument("acdac")
yoko = container.load_instrument("yoko")
keithely = container.load_instrument("keithley")
li = container.load_instrument("lockin")
# ppms = container.load_instrument("ppms") # update PPMS PC ethernet IP address to 169.254.x.x 

Connected to: BARRERA ACDAC02 (serial:AD9106, firmware:2.00) in 3.10s
Connected to: YOKOGAWA 765611 (serial:90ZC43353, firmware:1.10) in 0.02s
Connected to: KEITHLEY INSTRUMENTS DMM6500 (serial:04608216, firmware:1.7.12b) in 0.01s
Connected to: Stanford_Research_Systems SR865A (serial:005296, firmware:V1.51) in 0.09s


In [7]:
# TODO: aliases for parameters. Update labels to reflect physical quantities for following cell

# vg = yoko.channel1.voltage 
# vg.label = "Gate Voltage"
  
indep_params = []
dep_params = []

In [ ]:
# measurement config for analysis 
mc = {}
for param in indep_params + dep_params:
    mc[param.full_name] = param.label

s = sweep.Station(basedir=datapath, measurement_config=mc)
for param in indep_params + dep_params:
    s.fp(param)

s.add_comment("ENTER A COMMENT HERE")

In [ ]:
# instrument setups

# vg(0.1) ... 

In [1]:
# TODO: add parameters to live plot here 

# s.plot("time", li.R)
# s.plot(vg, li.R)

In [ ]:
# Run experiment 
# START = 0
# STOP = 1
# SAMPLES = 20
# INT_TIME = 0.4

# s.watch(delay = INT_TIME, max_duration=10)
# setpoints = np.linspace(START, STOP, SAMPLES)
# res = s.sweep(vg, setpoints, delay=INT_TIME)

In [ ]:
# Look at result metadate for verification 

# res.metadata